#  Regra dos Cinco

Uma das principais áreas em que a quiminformática é amplamente utilizada é a química medicinal. Sendo o desafio da química medicinal o desenvolvimento (muitas vezes, chamado incorretamente de "descoberta") de fármacos, é natural que em diversos momentos os pesquisadores tenham se perguntado: quais propriedades fazem com que uma molécula seja (ou não) um fármaco?

De fato, essa questão não tem uma resposta clara, tendo em vista, por exemplo, a existência de diversas substâncias utilizadas como fármacos no passado e que hoje são proibidas ([exemplos](https://en.wikipedia.org/wiki/List_of_withdrawn_drugs)), ou mesmo substâncias cujo uso é permitido em um país e proibido em outro. 

Ainda assim, desde que o desenvolvimento de fármacos passou a usar de estratégias mais racionais, mostrou-se necessário encontrar alguma maneira de selecionar compostos mais promissores, tendo em vista o (praticamente) infinito número de substâncias que podem ser sintetizadas e o alto custo do desenvolvimento de um fármaco.

Em 1997, Lipinski, Lombardo, Dominy e Feeney avaliaram uma série de fármacos aprovados, administrados por via oral, e observaram que a maior parte apresentava propriedades físico-químicas dentro de certos limites. São elas:

- Massa molecular relativa (MW) ≤ 500
- Logaritmo do coeficiente de partição octanol/água (logP) ≤ 5
- Número de aceptores de ligação de hidrogênio ≤ 10
- Número de doadores de ligação de hidrogênio ≤ 5

O grupo chamou essa tendência de **Regra dos Cinco** (*Rule of Five*) pois os limites observados são todos múltiplos de cinco. Hoje, o termo "Regra de Lipinski" é também usado. Foi sugerido que compostos bioativos que seguem a Regra dos Cinco têm mais chance de eventualmente serem aprovados como fármacos do que compostos que quebram uma ou mais dessas regras. Isso é uma consequência da influência desses descritores em propriedades relevantes para fármacos, como absorção por via oral, permeabilidade e solubilidade. Por exemplo, compostos com logP > 5, sendo mais lipofílicos, podem apresentar problemas de solubilidade no plasma sanguíneo.

Vejamos abaixo como verificar se um composto segue a Regra dos Cinco de Lipinski usando o RDKit:

In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [2]:
# Função para calcular os valores da regra dos cinco
def rule_of_five(smi):
    print()
    print("Regra dos Cinco de Lipinski")
    
    mol = Chem.MolFromSmiles(smi)
    d = ["MolLogP", "MolWt", "NumHAcceptors", "NumHDonors"]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator(d)
    values = calc.CalcDescriptors(mol)

    violations = 0
    if values[0] > 5:
        violations += 1
        print("    logP calculado > 5")
    if values[1] > 500:
        violations += 1
        print("    Massa molecular relativa > 500")
    if values[2] > 10:
        violations += 1
        print("    Número de aceptores de ligação de hidrogênio > 10")
    if values[3] > 5:
        violations += 1
        print("    Número de doaores de ligação de hidrogênio > 5")
    if violations:
        print(f"    Total de violações: {violations}")
    else: 
        print("    Nenhuma violação")
    print()

In [3]:
# SMILES do composto a ser verificado
smi = "COc1ccc2c(c1)nc([nH]2)S(=O)Cc1ncc(c(c1C)OC)C"  # exemplo: omeprazol

rule_of_five(smi)


Regra dos Cinco de Lipinski
    Nenhuma violação



A Regra dos Cinco parece ter funcionado para o omeprazol, um fármaco utilizado para tratar úlceras. 

Porém, seria essa uma regra absoluta para identificar fármacos (e não-fármacos)? 

É fácil verificar que não. Vejamos abaixo dois exemplos: taxol e atrazina.

In [4]:
taxol = "CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)C(C(C5=CC=CC=C5)NC(=O)C6=CC=CC=C6)O)O)OC(=O)C7=CC=CC=C7)(CO4)OC(=O)C)O)C)OC(=O)C"
rule_of_five(taxol)


Regra dos Cinco de Lipinski
    Massa molecular relativa > 500
    Número de aceptores de ligação de hidrogênio > 10
    Total de violações: 2



In [5]:
atrazina = "CCNc1nc(Cl)nc(NC(C)C)n1"
rule_of_five(atrazina)


Regra dos Cinco de Lipinski
    Nenhuma violação



Vemos que o taxol, um fármaco aprovado para o tratamento de cânceres, não segue a Regra dos Cinco, enquanto a atrazina, um agroquímico que não é usado como fármaco, não viola nenhuma regra.

A Regra dos Cinco é, dessa forma, constante alvo de críticas pela comunidade científica. Por exemplo, [um trabalho de 2015](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4342520/) indica que somente 50% dos fármacos orais aprovados seguiram todas as regras. Fármacos altamente relevantes, como atorvastatina (Lipitor) e montelukast (Singulair) também quebram mais de uma das regras, e uma nova classe de compostos relativamente "grandes", denominados PROTACs, vem sendo cada vez mais exploradas. Esse espaço químico de compostos relevantes na química medicinal que quebram a Regra dos Cinco é chamado na literatura de “*beyond rule of five*” (bRo5).

Em resumo, mesmo não sendo uma lei absoluta, a Regra ainda pode ser válida como guia, e deve ser quebrada somente quando houver uma boa razão.

# Outras regras e filtros

Outras regras derivadas de observações de grandes números de dados surgiram com o tempo, muitas delas sendo usadas como "filtros" em processos de triagem virtual de compostos. Seu uso mais comum consiste em partir de um banco de dados com milhões de estruturas e selecionar apenas as que se adequam a um conjunto de propriedades, o que, espera-se, aumentará a chance de sucesso no *pipeline* de desenvolvimento de fármacos. Alguns exemplos de filtros são apresentados a seguir:

**Filtro de Veber**: biodisponibilidade (absorção oral). [Fonte](https://doi.org/10.1021/jm020017n)
- Área polar superficial topológica (TPSA) ≤ 140 Å²
- Número de ligações rotacionáveis ≤ 10

**Filtro de Egan**: biodisponibilidade (absorção oral). [Fonte](https://doi.org/10.1021/jm000292e)
- logP ≤ 5.88 (o descritor AlogP98 foi usado)
- TPSA ≤ 131.6 Å²

**Filtro _beyond rule of five_**: biodisponibilidade (absorção oral). [Fonte](https://doi.org/10.1016/j.chembiol.2014.08.013)
- MW ≤ 1000
- −2 ≤ clogP ≤ 10
- HBD ≤ 6
- HBA ≤ 15
- PSA ≤ 250 Å²
- RotB ≤ 20

**Filtro de Ghose**: fármaco-similariadade (*drug-likeness*) [Fonte](https://pubs.acs.org/doi/full/10.1021/cc9800071)
- 160 < MW < 480
- −0.4 < logP < 5.6
- 20 < número de átomos < 70
- 40 < refratividade molar (MolMR no RDKit) < 130

**Índice de desejabilidade (*Desirability index*)**: fármaco-similariadade (*drug-likeness*). [Fonte](https://doi.org/10.1038%2Fnchem.1243)
- QED: valor que leva em conta diversas propriedades físico-químicas e a presença de alertas (PAINS). Varia de zero (todas as propriedades são desfavoráveis) a um (todas as propriedades favoráveis)

**Regra dos três**: para desenvolvimento de fármacos baseado em fragmentos. [Fonte] (https://www.sciencedirect.com/science/article/abs/pii/S1359644603028319)
- MW ≤ 300
- logP ≤ 3
- HBD ≤ 3
- HBA ≤ 3

**Filtro de Van der Waterbeemd**: permeabilidade na barreira hematoencefálica. [Fonte](https://onlinelibrary.wiley.com/doi/abs/10.1002/9783527623860.ch5)
- MW ≤ 450
- TPSA ≤ 90 Å²

**Filtro de Murcko**: permeabilidade na barreira hematoencefálica. [Fonte](https://doi.org/10.1021/jm990017w)
- 200 ≤ MW ≤ 400
- logP ≤ 5.2
- HBA ≤ 4
- HBD ≤ 3
- Número de ligações rotacionáveis ≤ 7

Caso deseje implementar algum desses filtros, basta adaptar a função `rule_of_five()` apresentada acima e incluir os descritores desejados. Veja abaixo uma lista contendo os descritores relevantes disponíveis no RDKit: 

In [6]:
d = ["FractionCSP3",
     "HeavyAtomCount", 
     "MolLogP", 
     "MolMR", 
     "MolWt", 
     "NumAromaticRings", 
     "NumHAcceptors", 
     "NumHDonors", 
     "NumRotatableBonds", 
     "TPSA",
     "qed"]

Infelizmente, outras propriedades relevantes para biodisponibilidade, como logD e pK<sub>a</sub>, não podem ser calculadas pelo RDKit.

**Algumas fontes consultadas**

- [Molecular Filters in Medicinal Chemistry](https://www.mdpi.com/2673-8392/3/2/35)
- [Trends in Molecular Properties, Bioavailability, and Permeability across the Bayer Compound Collection](https://pubs.acs.org/doi/10.1021/acs.jmedchem.2c01577)
- [Two Decades under the Influence of the Rule of Five and the Changing Properties of Approved Oral Drugs](https://pubs.acs.org/doi/10.1021/acs.jmedchem.8b00686)

Veja também o *Notebook* [Eficiência do ligante e métricas de qualidade](https://github.com/rflameiro/Python_e_Quiminformatica/blob/main/Quiminformatica/Efici%C3%AAncia%20do%20ligante%20e%20m%C3%A9tricas%20de%20qualidade.ipynb)